In [183]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import re
import nltk
import spacy
import os
nlp = spacy.load("en_core_web_md")




def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 2
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()
    text = re.sub('\n', ' ', text)
    

    fp.close()
    device.close()
    retstr.close()
    return text


def remove_SpeChar(text):
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text, flags=re.MULTILINE)
    text = re.sub('\S*@\S*\s?', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub(r'[^\w]', ' ', text)
    text = re.sub('[!"#$%&()*+,./:;<=>?@[\]^_`{|}~‘’•]', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join(text.split())
    return text


def remove_StopWords(text):
    sentence = nlp(text)
    filtered_sentence = ''
    token_list = []
    for token in sentence : 
        token_list.append(token.text)
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False :
            filtered_sentence += ' '+word
    return filtered_sentence


def get_lem(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

def remove_Ent(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'DATE' : 
            text = re.sub(ent.text,' ',text)
        elif ent.label_ == 'PERSON' : 
            text = re.sub(ent.text,' ',text)
        elif ent.label_ == 'GPE' : 
            text = re.sub(ent.text,' ',text)
        
    return text

def get_low(text):
    return text.lower()
            

def main_(path):
    text = convert_pdf_to_txt(path)
    text = remove_SpeChar(text)
    text = remove_StopWords(text)
    text = remove_Ent(text)
    text = get_lem(text)
    text = get_low(text)
    return text
    

In [223]:
import uuid
List = []
def insert_into(path):
    id = uuid.uuid1()
    path_selector(path)
    for i in path_selector(path):
            List.append({'id' : id ,'name' : re.sub('[\d+ .pdf !"#$%&()*+,./:;<=>?@[\]^_`{|}~‘’• CV cv ]','',i.get('name')) , 'text' : main_(i.get('path'))})
    return List

In [227]:
def path_selector(path):
    """ this function will return a list of all files paths with pdf format"""
    L=[]
    for filename in os.listdir(path):
        if filename.endswith('.pdf'):
            L.append({'name': filename, 'path': os.path.join(path, filename)})
    return L

In [225]:
insert_into(r'..\stage\Resume')

[{'id': UUID('3dc7ff1e-d4d1-11ea-bd4f-b20e92867be1'),
  'name': 'Jonas',
  'text': '  jonas experience          accountant citco fund services singapore pte ltd position level hedge fund relate account financial consultant axa life insurance singapore pte ltd position level assisted client plan ahead financial planning model provide consistent support insurance coverage investment portfolio specialized life general insurance product knowledge focus need base sell go extra mile achieve high customer satisfaction level co partner finance consultant e commerce position level maintained company s bookkeeping ensure adherence government cpf policie iras guideline adapt different business platform openness innovative idea display excellent customer service lead long term customer satisfaction continuous support prompt reply postage handling conduct periodic market research maintain competitiveness improve product quality user friendliness credit approval ocbc bank limited position level    m

In [226]:
def tokenizer(text):
    doc = nlp(text)
    L=[]
    for token in doc : 
        if re.search(' +', token.text)==None:
            L.append(token.text)
    return L

In [135]:
tokenizer(List[0].get('text'))

['jonas',
 'experience',
 'accountant',
 'citco',
 'fund',
 'services',
 'singapore',
 'pte',
 'ltd',
 'position',
 'level',
 'hedge',
 'fund',
 'relate',
 'account',
 'financial',
 'consultant',
 'axa',
 'life',
 'insurance',
 'singapore',
 'pte',
 'ltd',
 'position',
 'level',
 'assisted',
 'client',
 'plan',
 'ahead',
 'financial',
 'planning',
 'model',
 'provide',
 'consistent',
 'support',
 'insurance',
 'coverage',
 'investment',
 'portfolio',
 'specialized',
 'life',
 'general',
 'insurance',
 'product',
 'knowledge',
 'focus',
 'need',
 'base',
 'sell',
 'go',
 'extra',
 'mile',
 'achieve',
 'high',
 'customer',
 'satisfaction',
 'level',
 'co',
 'partner',
 'finance',
 'consultant',
 'e',
 'commerce',
 'position',
 'level',
 'maintained',
 'company',
 's',
 'bookkeeping',
 'ensure',
 'adherence',
 'government',
 'cpf',
 'policie',
 'iras',
 'guideline',
 'adapt',
 'different',
 'business',
 'platform',
 'openness',
 'innovative',
 'idea',
 'display',
 'excellent',
 'customer'

In [50]:
import sys
import pymongo
from pymongo import MongoClient
Client = MongoClient("mongodb+srv://**********@nlpintern.9ghch.mongodb.net/Resume?retryWrites=true&w=majority")
Db = Client.NLPIntern
Col= Db.Resume
Col.insert_many(List)